In [97]:
from lxml import etree
from pprint import pprint

In [124]:
with open('template.xml', 'rb') as f:
    data = f.read()

root = etree.fromstring(data)
elements = root.xpath("//mxCell[string-length(@id) > 1 and string-length(@value) < 20]")


In [125]:
get_id = etree.XPath("@id")
get_value = etree.XPath("@value")
get_x = etree.XPath("mxGeometry/@x")
get_y = etree.XPath("mxGeometry/@y")
get_w = etree.XPath("mxGeometry/@width")
get_h = etree.XPath("mxGeometry/@height")
get_style = etree.XPath("@style")

class_dict = {
    "#dae8fc": "window",
    "#ffe6cc": "sidebar",
    "#fff2cc": "title",
    "#d5e8d4": "container",
    "#e1d5e7": "column",
    "#f8cecc": "divider"
}

def get_class(el: etree.Element) -> str:
    style = get_style(el)[0]
    style_lst = style.split(";")
    for item in style_lst:
        if "=" in item:
            key, val = item.split("=")
            if key == "fillColor":
                return class_dict[val]
        
    return "component"

In [126]:
def get_data(el: etree.Element) -> dict:
    width = float(get_w(el)[0])
    height = float(get_h(el)[0])
    size = width*height

    return {
            "id": get_id(el)[0],
            "value": get_value(el)[0].lower(),
            "x": float(get_x(el)[0]),
            "y": float(get_y(el)[0]),
            "w": width,
            "h": height,
            "size": size,
            "class": get_class(el),
        }

def rescale(lst: list, x: float, y: float, w: float, h: float) -> list:
    for i in range(len(lst)):
        lst[i]['x'] -= x
        lst[i]['y'] -= y
        lst[i]['w'] /= w
        lst[i]['h'] /= h

In [127]:
element_data = [get_data(el) for el in elements]
element_data = sorted(element_data, key=lambda x: x["size"], reverse=True)
canvas = element_data[0]
#rescale(element_data, canvas['x'], canvas['y'], canvas['w'], canvas['h'])

In [128]:
def contains(el1: dict,el2: dict) -> bool:
    if el1['x'] <= el2['x'] and el1['y'] <= el2['y']:
        if el2['x'] + el2['w'] <= el1['x'] + el1['w'] and el2['y'] + el2['h'] <= el1['y'] + el1['h']:
            return True
    return False


In [129]:
for i in range(len(element_data)):
    element_data[i]["children"] =  []

i = len(element_data)-2
while i > -1 and len(element_data) > 1:
    for j in range(len(element_data)-1, i, -1):
        if contains(element_data[i], element_data[j]):
            element_data[i]["children"].append(element_data[j])
            element_data.pop(j)
    i -= 1

{'id': 'VK38lUuWJpWfB77Lczag-3', 'value': '', 'x': 504.45346321736554, 'y': 361.14300000000003, 'w': 116.49099066309846, 'h': 110.72028566666665, 'size': 12897.91576381126, 'class': 'column', 'children': []} {'id': 'GI5be8NEMBNBnBu12xkf-14', 'value': 'image', 'x': 513.6295480225989, 'y': 364.60333333333335, 'w': 98.1491525423729, 'h': 103.80999999999999, 'size': 10188.86352542373, 'class': 'component', 'children': []}
{'id': 'VK38lUuWJpWfB77Lczag-1', 'value': '', 'x': 243.8485221528397, 'y': 361.14300000000003, 'w': 116.49099066309846, 'h': 110.72028566666665, 'size': 12897.91576381126, 'class': 'column', 'children': []} {'id': 'GI5be8NEMBNBnBu12xkf-13', 'value': 'image', 'x': 251.8984745762712, 'y': 364.60333333333335, 'w': 98.1491525423729, 'h': 103.80999999999999, 'size': 10188.86352542373, 'class': 'component', 'children': []}
{'id': 'VK38lUuWJpWfB77Lczag-4', 'value': '', 'x': 242.7292774308653, 'y': 499.56152383333335, 'w': 116.49099066309846, 'h': 110.72028566666665, 'size': 1289

In [130]:
element_data[0]

{'id': 'GI5be8NEMBNBnBu12xkf-1',
 'value': '',
 'x': 40.0,
 'y': 240.0,
 'w': 760.0,
 'h': 400.0,
 'size': 304000.0,
 'class': 'window',
 'children': [{'id': 'VK38lUuWJpWfB77Lczag-11',
   'value': 'divider',
   'x': 240.0,
   'y': 314.57,
   'w': 540.0,
   'h': 15.43,
   'size': 8332.2,
   'class': 'component',
   'children': []},
  {'id': 'REH8p-8bh4SVzu3RKBWu-2',
   'value': '',
   'x': 660.0,
   'y': 340.0,
   'w': 130.0,
   'h': 290.0,
   'size': 37700.0,
   'class': 'column',
   'children': [{'id': 'GI5be8NEMBNBnBu12xkf-9',
     'value': 'image',
     'x': 670.4545454545452,
     'y': 501.58666666666664,
     'w': 109.09090909090907,
     'h': 121.11166666666665,
     'size': 13212.181818181813,
     'class': 'component',
     'children': []},
    {'id': 'GI5be8NEMBNBnBu12xkf-8',
     'value': 'image',
     'x': 670.4545454545452,
     'y': 345.87166666666667,
     'w': 109.09090909090907,
     'h': 138.4133333333333,
     'size': 15099.636363636357,
     'class': 'component',
   

In [131]:
element_tree = element_data[0]

In [132]:
pprint(element_tree)

{'children': [{'children': [],
               'class': 'component',
               'h': 15.43,
               'id': 'VK38lUuWJpWfB77Lczag-11',
               'size': 8332.2,
               'value': 'divider',
               'w': 540.0,
               'x': 240.0,
               'y': 314.57},
              {'children': [{'children': [],
                             'class': 'component',
                             'h': 121.11166666666665,
                             'id': 'GI5be8NEMBNBnBu12xkf-9',
                             'size': 13212.181818181813,
                             'value': 'image',
                             'w': 109.09090909090907,
                             'x': 670.4545454545452,
                             'y': 501.58666666666664},
                            {'children': [],
                             'class': 'component',
                             'h': 138.4133333333333,
                             'id': 'GI5be8NEMBNBnBu12xkf-8',
                     

In [ ]:
class_dict = {
    "#dae8fc": "window",
    "#ffe6cc": "sidebar",
    "#fff2cc": "title",
    "#d5e8d4": "container",
    "#e1d5e7": "column",
    "#f8cecc": "divider"
}

In [137]:
TAB = r"\t"

def recursive_streamlit(el_dict, col_count=0, indent = 0):
    children = el_dict['children']
    if not len(children):
        if el_dict['class'] == "component":
            if el_dict['value'] == "image":
                return [f'{indent * TAB}st.image("default.png")']
            else:
                return [f'{indent * TAB}st.{el_dict["value"]}(key={el_dict["id"]})']
        elif el_dict['class'] == "title":
            return [f'{indent * TAB}st.title({el_dict["value"]})']
        else:
            print("The element below was expecting to have at least one child element.")
            print(el_dict)
            return []

    if el_dict['class'] == "window":
        code = [f'{indent * TAB}st.set_page_config(layout="wide")']
    elif el_dict['class'] == "container":
        code = [f'{indent * TAB}with st.container()']
        indent += 1
    else:
        code = []

    for child in children:
        print(child)
        try:
            if child['class'] != "column":
                code += recursive_streamlit(child, col_count, indent)
                print("gotcha!")
            else:
                columns = [el for el in children if el['class']=='column']
                vars = ", ".join([f"col{col_count+i}" for i in range(len(columns))])
                code += [f"{indent * TAB}{vars} = st.columns({str([col['w'] for col in columns])})"]
                for i, col in enumerate(columns):
                    code += [f"{indent * TAB}with col{i+col_count}:"]
                    code += recursive_streamlit(col, col_count+len(columns), indent+1)
                for col in columns:
                    children.remove(col)
        except Exception as e:
            print(e)
            print("columns already handled")

    return code
        

code = recursive_streamlit(element_tree)

{'id': 'VK38lUuWJpWfB77Lczag-11', 'value': 'divider', 'x': 240.0, 'y': 314.57, 'w': 540.0, 'h': 15.43, 'size': 8332.2, 'class': 'component', 'children': []}
gotcha!
{'id': 'GI5be8NEMBNBnBu12xkf-2', 'value': '', 'x': 40.0, 'y': 240.0, 'w': 175.3846153846154, 'h': 400.0, 'size': 70153.84615384616, 'class': 'sidebar', 'children': [{'id': 'GI5be8NEMBNBnBu12xkf-6', 'value': 'slider', 'x': 54.61538461538461, 'y': 425.7142857142858, 'w': 146.15384615384616, 'h': 28.571428571428573, 'size': 4175.824175824177, 'class': 'component', 'children': []}, {'id': 'GI5be8NEMBNBnBu12xkf-5', 'value': 'slider', 'x': 54.61538461538461, 'y': 382.8571428571429, 'w': 146.15384615384616, 'h': 28.571428571428573, 'size': 4175.824175824177, 'class': 'component', 'children': []}, {'id': 'GI5be8NEMBNBnBu12xkf-4', 'value': 'slider', 'x': 54.61538461538461, 'y': 340.0, 'w': 146.15384615384616, 'h': 28.571428571428573, 'size': 4175.824175824177, 'class': 'component', 'children': []}, {'id': 'GI5be8NEMBNBnBu12xkf-17', 

In [139]:
element_tree

{'id': 'GI5be8NEMBNBnBu12xkf-1',
 'value': '',
 'x': 40.0,
 'y': 240.0,
 'w': 760.0,
 'h': 400.0,
 'size': 304000.0,
 'class': 'window',
 'children': [{'id': 'VK38lUuWJpWfB77Lczag-11',
   'value': 'divider',
   'x': 240.0,
   'y': 314.57,
   'w': 540.0,
   'h': 15.43,
   'size': 8332.2,
   'class': 'component',
   'children': []},
  {'id': 'GI5be8NEMBNBnBu12xkf-2',
   'value': '',
   'x': 40.0,
   'y': 240.0,
   'w': 175.3846153846154,
   'h': 400.0,
   'size': 70153.84615384616,
   'class': 'sidebar',
   'children': [{'id': 'GI5be8NEMBNBnBu12xkf-6',
     'value': 'slider',
     'x': 54.61538461538461,
     'y': 425.7142857142858,
     'w': 146.15384615384616,
     'h': 28.571428571428573,
     'size': 4175.824175824177,
     'class': 'component',
     'children': []},
    {'id': 'GI5be8NEMBNBnBu12xkf-5',
     'value': 'slider',
     'x': 54.61538461538461,
     'y': 382.8571428571429,
     'w': 146.15384615384616,
     'h': 28.571428571428573,
     'size': 4175.824175824177,
     'cla

In [138]:
print("\n".join(code))

st.set_page_config(layout="wide")
st.divider(key=VK38lUuWJpWfB77Lczag-11)
st.slider(key=GI5be8NEMBNBnBu12xkf-6)
st.slider(key=GI5be8NEMBNBnBu12xkf-5)
st.slider(key=GI5be8NEMBNBnBu12xkf-4)
st.image("default.png")
